In [9]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Mixed Precision Training 설정 (최신 API 사용)
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# 모델 저장 폴더 설정
save_dir = '/home/heechun/final_ws/Pose_Table/1118_upgrade_table'
os.makedirs(save_dir, exist_ok=True)  # 폴더가 없을 경우 생성

# 학습용 데이터 증강을 위한 ImageDataGenerator 설정
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,             # 각도 범위 축소
    width_shift_range=0.4,         # 이동 범위 축소
    height_shift_range=0.4,        # 이동 범위 축소
    shear_range=0.3,              # 왜곡 정도 축소
    zoom_range=[0.6, 1.5],         # 줌 범위 축소
    horizontal_flip=True,
    brightness_range=[0.6, 1.5],   # 밝기 범위 축소
    channel_shift_range=40.0,      # 채널 이동 범위 축소
    fill_mode='nearest',
    validation_split=0.2
)


# 검증용 데이터 스케일링만 적용
validation_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2
)

# 학습용 데이터 로드
train_generator = train_datagen.flow_from_directory(
    '/home/heechun/final_ws/TableCheck/record_table/data/train/',
    target_size=(224, 224),          
    batch_size=32,                   
    class_mode='binary',             
    subset='training'                
)

# 검증용 데이터 로드
validation_generator = validation_datagen.flow_from_directory(
    '/home/heechun/final_ws/TableCheck/record_table/data/train/',
    target_size=(224, 224),          
    batch_size=32,                   
    class_mode='binary',             
    subset='validation'              
)

# MobileNetV2 기반 모델 구성
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # 모든 레이어를 미세 조정 가능하도록 설정

# 특정 레이어까지만 고정
fine_tune_at = 40
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False


# 모델 정의 (Dense 레이어 최적화 및 Swish 활성화 함수 사용)
model = tf.keras.models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.4),
    layers.BatchNormalization(),
    layers.Dense(256, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Dropout(0.4),
    layers.BatchNormalization(),
    layers.Dense(128, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Dropout(0.4),
    layers.BatchNormalization(),
    layers.Dense(64, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Dropout(0.4),
    layers.Dense(1, activation='sigmoid', dtype='float32')  # Mixed Precision 사용 시 dtype 명시 필요
])

# AdamW 옵티마이저 설정
optimizer = tf.keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-5)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 체크포인트 설정
checkpoint_path = os.path.join(save_dir, '1118_model_01.keras')
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1
)

# Early Stopping 설정
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=15, restore_best_weights=True, verbose=1
)

# 학습률 감소 스케줄러 설정
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=5, verbose=1
)

# 학습 로그 저장 (TensorBoard 사용)
log_dir = os.path.join(save_dir, 'logs_1118_01')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

# 모델 학습
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=80,  # 에포크 수 증가
    callbacks=[model_checkpoint, tensorboard_callback, early_stopping, lr_scheduler]
)


Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Epoch 1/80


/home/heechun/venv/md_ven/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step - accuracy: 0.5255 - loss: 1.5240
Epoch 1: val_accuracy improved from -inf to 0.70750, saving model to /home/heechun/final_ws/Pose_Table/1118_upgrade_table/1118_model_01.keras
50/50 ━━━━━━━━━━━━━━━━━━━━ 38s 487ms/step - accuracy: 0.5265 - loss: 1.5228 - val_accuracy: 0.7075 - val_loss: 1.3007 - learning_rate: 1.0000e-04
Epoch 2/80
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 376ms/step - accuracy: 0.5985 - loss: 1.3892
Epoch 2: val_accuracy improved from 0.70750 to 0.73000, saving model to /home/heechun/final_ws/Pose_Table/1118_upgrade_table/1118_model_01.keras
50/50 ━━━━━━━━━━━━━━━━━━━━ 24s 418ms/step - accuracy: 0.5991 - loss: 1.3883 - val_accuracy: 0.7300 - val_loss: 1.2252 - learning_rate: 1.0000e-04
Epoch 3/80
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 391ms/step - accuracy: 0.6688 - loss: 1.2888
Epoch 3: val_accuracy did not improve from 0.73000
50/50 ━━━━━━━━━━━━━━━━━━━━ 25s 428ms/step - accuracy: 0.6694 - loss: 1.2877 - val_accuracy: 0.7225 - val_loss: 1.1591 - lea

In [1]:
!tensorboard --logdir=/home/heechun/final_ws/Pose_Table/1118_upgrade_table/logs_1118_02

2024-11-19 09:24:06.568999: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-19 09:24:06.585765: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-19 09:24:06.590679: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-19 09:24:06.603695: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-19 09:24:07.895946: W tensorflow/compiler/tf2

In [10]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Mixed Precision Training 설정 (최신 API 사용)
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# 모델 저장 폴더 설정
save_dir = '/home/heechun/final_ws/Pose_Table/1118_upgrade_table'
os.makedirs(save_dir, exist_ok=True)  # 폴더가 없을 경우 생성

# 학습용 데이터 증강을 위한 ImageDataGenerator 설정
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,             # 각도 범위 축소
    width_shift_range=0.4,         # 이동 범위 축소
    height_shift_range=0.4,        # 이동 범위 축소
    shear_range=0.3,              # 왜곡 정도 축소
    zoom_range=[0.6, 1.5],         # 줌 범위 축소
    horizontal_flip=True,
    brightness_range=[0.6, 1.5],   # 밝기 범위 축소
    channel_shift_range=40.0,      # 채널 이동 범위 축소
    fill_mode='nearest',
    validation_split=0.2
)


# 검증용 데이터 스케일링만 적용
validation_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2
)

# 학습용 데이터 로드
train_generator = train_datagen.flow_from_directory(
    '/home/heechun/final_ws/TableCheck/record_table/data/train/',
    target_size=(224, 224),          
    batch_size=32,                   
    class_mode='binary',             
    subset='training'                
)

# 검증용 데이터 로드
validation_generator = validation_datagen.flow_from_directory(
    '/home/heechun/final_ws/TableCheck/record_table/data/train/',
    target_size=(224, 224),          
    batch_size=32,                   
    class_mode='binary',             
    subset='validation'              
)

# MobileNetV2 기반 모델 구성
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # 모든 레이어를 미세 조정 가능하도록 설정

# 특정 레이어까지만 고정
fine_tune_at = 20
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False


# 모델 정의 (Dense 레이어 최적화 및 Swish 활성화 함수 사용)
model = tf.keras.models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.BatchNormalization(),
    layers.Dense(256, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.BatchNormalization(),
    layers.Dense(128, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.BatchNormalization(),
    layers.Dense(64, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid', dtype='float32')  # Mixed Precision 사용 시 dtype 명시 필요
])

# AdamW 옵티마이저 설정
optimizer = tf.keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-5)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 체크포인트 설정
checkpoint_path = os.path.join(save_dir, '1118_model_02.keras')
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1
)

# Early Stopping 설정
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=15, restore_best_weights=True, verbose=1
)

# 학습률 감소 스케줄러 설정
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=5, verbose=1
)

# 학습 로그 저장 (TensorBoard 사용)
log_dir = os.path.join(save_dir, 'logs_1118_02')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

# 모델 학습
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=80,  # 에포크 수 증가
    callbacks=[model_checkpoint, tensorboard_callback, early_stopping, lr_scheduler]
)


Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Epoch 1/80
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 354ms/step - accuracy: 0.5060 - loss: 1.5631
Epoch 1: val_accuracy improved from -inf to 0.71250, saving model to /home/heechun/final_ws/Pose_Table/1118_upgrade_table/1118_model_02.keras
50/50 ━━━━━━━━━━━━━━━━━━━━ 36s 460ms/step - accuracy: 0.5069 - loss: 1.5619 - val_accuracy: 0.7125 - val_loss: 1.3329 - learning_rate: 1.0000e-04
Epoch 2/80
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 363ms/step - accuracy: 0.5574 - loss: 1.4798
Epoch 2: val_accuracy improved from 0.71250 to 0.73000, saving model to /home/heechun/final_ws/Pose_Table/1118_upgrade_table/1118_model_02.keras
50/50 ━━━━━━━━━━━━━━━━━━━━ 23s 402ms/step - accuracy: 0.5578 - loss: 1.4791 - val_accuracy: 0.7300 - val_loss: 1.2792 - learning_rate: 1.0000e-04
Epoch 3/80
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 363ms/step - accuracy: 0.6069 - loss: 1.4107
Epoch 3: val_accuracy improved from 0.73000 to 0.73500, saving model to /home/he